In [43]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import sqlalchemy

from xml.etree import ElementTree

import mercante

engine = sqlalchemy.create_engine(
    'mysql+mysqlconnector://ivan@localhost:3306/mercante')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
from collections import Counter

count_objetos = Counter()


In [45]:


lista_arquivos = os.listdir(mercante.MERCANTE_DIR)
for arquivo in lista_arquivos:
    xtree = ElementTree.parse(os.path.join(mercante.MERCANTE_DIR, arquivo))
    xroot = xtree.getroot()
    objetos = []
    for node in xroot:
        classe = mercante.classes.get(node.tag)
        if classe:
            count_objetos[classe]+=1
            # print(node.tag, node.text)
            objeto = classe()
            objeto._parse_node(node)
            # objeto = Conhecimento()
            # objeto._parse_node(node)
            # print(objeto)
            objetos.append(objeto._to_dict())
    df = pd.DataFrame(objetos)
    try:
        df.to_sql(node.tag, engine, if_exists='append')
    except Exception as err:
        print('Erro ocorrido no arquivo %s.' % arquivo)
        print(err, exc_info=True)
    

In [46]:
print('%d arquivos processados como %d objetos' % (len(lista_arquivos), sum(count_objetos.values())))

187 arquivos processados como 53959 objetos


In [48]:
for arquivo in lista_arquivos:
    xtree = ElementTree.parse(os.path.join(mercante.MERCANTE_DIR, arquivo))
    xroot = xtree.getroot()
    objetos = []
    for node in xroot:
        classe = mercante.classes_em_lista.get(node.tag)
        if classe:
            last_class_name = classe
            classe_pai = mercante.classes.get(node.tag)
            objeto_pai = classe_pai()
            objeto_pai._parse_node(node)
            tag_classe = classe.tag
            for subnode in node.findall(tag_classe):
                count_objetos[classe]+=1
                objeto = classe(objeto_pai)
                objeto._parse_node(subnode)
                objetos.append(objeto._to_dict())
            classe = mercante.classes_em_lista.get(node.tag)
    if objetos and len(objetos) > 0:
        df = pd.DataFrame(objetos)
        classname = objeto.__class__.__name__
        try:
            df.to_sql(classname, engine, if_exists='append')
        except Exception as err:
            print('Erro ocorrido no arquivo %s.' % arquivo)
            print(err)
    

In [49]:
count_objetos

Counter({mercante.Conhecimento: 14891,
         mercante.ItemCarga: 36825,
         mercante.Manifesto: 2235,
         mercante.ExclusaoEscala: 8,
         mercante.NCMItemCarga: 85058,
         mercante.ConteinerVazio: 12862})